# Important: Add competition dataset as input dataset first for this notebook to work

In [2]:
!pip install -U bitsandbytes peft accelerate datasets sentencepiece wandb python-dotenv wtpsplit langchain
!pip install flash-attn --no-build-isolation
!pip install wtpsplit==2.1.1
!pip install syntok==1.4.4
!pip install omegaconf
!pip install wandb
!pip install --upgrade transformers trl
!pip install pandas numpy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 26.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.3/336.3 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 71.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8

In [3]:
!git clone https://github.com/Reennon/gen-ai-nlp-lab-1

Cloning into 'gen-ai-nlp-lab-1'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 42 (delta 13), reused 29 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 185.39 KiB | 6.18 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [4]:
%cd gen-ai-nlp-lab-1
!ls

/kaggle/working/gen-ai-nlp-lab-1
README.md  notebooks  params  poetry.lock  pyproject.toml  src


In [5]:
import os
import torch
import pandas as pd

from langchain_core.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from huggingface_hub import login
from transformers import PreTrainedTokenizerBase, BitsAndBytesConfig
from tqdm import tqdm
from torch.utils.data import Dataset
from datasets import Dataset
from src.prompts.prompts import (NERPrompt1, NERPrompt2, NERPrompt3, NERPrompt4)
from src.prompts.examples import (
    ELEVEN_SHOT_EXAMPLES_DICT, 
    FIVE_SHOT_EXAMPLES_DICT, 
    THREE_SHOT_EXAMPLES_DICT, 
    ZERO_SHOT_EXAMPLES_DICT
)
from omegaconf import OmegaConf

from kaggle_secrets import UserSecretsClient

In [6]:
QUANTIZE_4BIT = True
# device   = "cuda:0"
device = "cuda" if torch.cuda.is_available() else "cpu"
device_ids = [0, 1]  # Assuming two T4 GPUs with IDs 0 and 1

In [7]:
parameters = OmegaConf.load("./params/aya_23_8b.yml")

In [8]:
login(UserSecretsClient().get_secret("HUGGINGFACE_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [26]:
checkpoint = "CohereForAI/aya-23-8b"
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
seq_length = parameters.baseline.max_new_tokens
tokenizer.model_max_length = seq_lenght
max_seq_length = seq_length
config = AutoConfig.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    config=config,
    quantization_config=quantization_config,
    torch_dtype="bfloat16",
    device_map="auto",  # Automatically map to GPUs
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
silver_train_df = pd.read_csv("/kaggle/input/genai-ucu-2024-lab1-preprocessed/silver_train.csv")
silver_test_df = pd.read_csv("/kaggle/input/genai-ucu-2024-lab1-preprocessed/silver_test.csv")

In [11]:
from src.prompts.prompts import BasePrompt

class NERPrompt1(BasePrompt):
    template: str = """Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей:
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей.

Нижче наведено приклади формату та стилю розпізнавання сутностей:
{examples}

Вхідний текст:
{text}

Спочатку ознайомся з прикладами, потім визнач усі сутності у наведеному тексті.
"""
    input_variables: list[str] = ["text", "examples"]

In [12]:
example_template: str = """
Вхідний текст:
{example_text}
Знайдені сутності:
{example_labels}
"""

def construct_prompt(
    prompt_template: PromptTemplate,
    few_shot_dict: dict[str, str], 
    text: str
) -> str:
    examples = "".join([
            example_template.format(
                example_text=example["text"],
                example_labels=example["labels"]
            ) for example in few_shot_dict
        ])
    prompt = prompt_template.format(
        examples=examples,
        text=text,
    )

    return prompt

prompt = construct_prompt(
    prompt_template=NERPrompt1().prompt_template,
    few_shot_dict=ELEVEN_SHOT_EXAMPLES_DICT,
    text=""
)
print(prompt)

Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей:
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей.

Нижче наведено приклади формату та стилю розпізнавання сутностей:

Вхідний текст:
український біз

In [13]:
tokenized_prompt = tokenizer.encode(prompt)
decoded_prompt = tokenizer.decode(tokenized_prompt)
prompt_len = len(tokenizer.tokenize(prompt))
print(decoded_prompt), prompt_len

Token indices sequence length is longer than the specified maximum sequence length for this model (1687 > 800). Running this sequence through the model will result in indexing errors


<BOS_TOKEN>Твоє завдання – виділити всі сутності у наданому тексті за наведеними категоріями та вивести їх у форматі JSON-списку:

Категорії сутностей:
- ART: артефакт (створений людиною предмет)
- DATE: дата (календарна дата, рік)
- DOC: документ (назви документів)
- JOB: посада (професійний титул, робоча позиція)
- LOG: місце (географічні об’єкти, назви країн, міст, річок тощо)
- MISC: різне (інші сутності, не підпадають під інші категорії)
- MON: гроші (сума, валюта)
- ORG: організація (установи, компанії, заклади)
- PCT: відсоток (число у відсотках)
- PERIOD: період (тривалість часу)
- PERS: особа (людські імена, прізвища)
- QUANT: кількість (числові значення)
- TIME: час (конкретний момент доби)

Формат відповіді: список об’єктів у JSON, кожен об’єкт має поля:
"label" – категорія сутності
"text" – фрагмент тексту сутності з оригінального тексту без змін

Не виводь дублікати знайдених сутностей.

Нижче наведено приклади формату та стилю розпізнавання сутностей:

Вхідний текст:
укра

(None, 1686)

In [14]:
def get_message_format(prompts):
  messages = []

  for p in prompts:
    messages.append(
        [{"role": "user", "content": p}]
      )

  return messages

In [29]:
out_labels = {}
max_seq_length = prompt_len + seq_length

for id in tqdm(silver_test_df.id.unique(), desc="Text progress"):
    inputs: list[str] = silver_test_df.loc[silver_test_df.loc[:, "id"] == id, "text"].to_list()
    inputs: list[str] = [construct_prompt(
        prompt_template=NERPrompt1().prompt_template,
        few_shot_dict=THREE_SHOT_EXAMPLES_DICT,
        text=input
    ) for input in inputs]
    print("inputs len")
    print([len(tokenizer.encode(input)) for input in inputs])
    print()
    inputs: list[dict[str, str]] = get_message_format(inputs)
    input_ids = tokenizer.apply_chat_template(
        inputs,
        tokenize=True,
        add_generation_prompt=True,
        padding=True,
        return_tensors="pt",
    )
    input_ids = input_ids.to(model.device)
    #input_ids = input_ids.to(model.device)
    prompt_padded_len = len(input_ids[0])
    # Generate corrections
    # Check if the model is wrapped in DataParallel
    gen_tokens = model.generate(
            input_ids,
            temperature=parameters.baseline.temperature,
            top_p=parameters.baseline.top_p,
            top_k=parameters.baseline.top_k,
            max_new_tokens=seq_length,
            do_sample=True,
    )

    gen_tokens = [
        gt[prompt_padded_len:] for gt in gen_tokens
    ]
    outputs: list[str] = tokenizer.batch_decode(
        gen_tokens,
        skip_special_tokens=True
    )
    # Join divided texts if any
    entities = ", ".join(outputs)
    print("output len")
    print(len(tokenizer.encode(entities)))
    print("output")
    print(id, entities)
    
    out_labels[id] = "".join(entities)

print("All texts extracted!")

silver_test_predictions_df = pd.DataFrame.from_dict([out_labels])
silver_test_predictions_df

Text progress:   0%|          | 0/169 [00:00<?, ?it/s]

inputs len
[1103, 908]



Text progress:   1%|          | 1/169 [04:15<11:56:13, 255.79s/it]

output len
705
output
e29896ab781b5dbb97ae3f3f7862fa681e9d70a5e63866024e2473b317a25637 Ось JSON-список, що містить виділені сутності та їхні категорії:

```json
[
    {
        "label": "ORG",
        "text": "департамент культури і туризму Кіровоградської ОДА"
    },
    {
        "label": "LOG",
        "text": "Моринці",
        "date": "1814 рік"
    },
    {
        "label": "LOG",
        "text": "Кирилівка",
        "date": "1929 рік"
    },
    {
        "label": "LOG",
        "text": "Звенигородський район"
    },
    {
        "label": "LOG",
        "text": "Україна"
    },
    {
        "label": "MISC",
        "text": "Шевченкові стежки"
    },
    {
        "label": "PERS",
        "text": "Тарас Шевченко",
        "date": "1814 рік"
    },
    {
        "label": "PERS",
        "text": "Катерина Бойко (мама поета)"
    },
    {
        "label": "ORG",
        "text": "музейний комплекс",
        "date": "1908 рік"
    },
    {
        "label": "ORG",
        "text": "лі

Text progress:   1%|          | 2/169 [08:46<12:16:45, 264.70s/it]

output len
716
output
d67655fe3fe45e95cd63613c2189fe86728293bd0b8d3cdf28598c5a63c651c7 Ось JSON-список, що містить виділені сутності та їхні категорії:

```json
[
    {
        "label": "ORG",
        "text": "Збройні Сили України"
    },
    {
        "label": "LOG",
        "text": "Криму",
        "date": "2014 р."
    },
    {
        "label": "LOG",
        "text": "Донбасі"
    },
    {
        "label": "MON",
        "text": "сотні мільйонів доларів"
    },
    {
        "label": "ORG",
        "text": "українська жіноча збірна з шахів"
    },
    {
        "label": "ORG",
        "text": "чоловіча збірна з шахів"
    },
    {
        "label": "LOG",
        "text": "Кіровоградщина"
    },
    {
        "label": "LOG",
        "text": "Павлівка Кремгесівського (Світловодського) району"
    },
    {
        "label": "LOG",
        "text": "імпровізований вуличний шаховий клуб “ у діда Миколи ”"
    },
    {
        "label": "LOG",
        "text": "Будинок культури"
    },
    {
 

Text progress:   1%|          | 2/169 [13:33<18:52:18, 406.81s/it]


KeyboardInterrupt: 

In [ ]:
import json

def location_baseline(text):
    text = text[0].lower() + text[1:]
    text = text.split(" ")
    text = [item for item in text if item[0].isupper()]
    text = [{"label": "LOC", "text": item} for item in text]
    text = json.dumps(text)
    return text


In [ ]:
df["entities"] = df["text"].apply(location_baseline)
df = df.drop(columns=["text"])
df.to_csv("submission.csv", index=False)